# Simulation of 2D data with g2opy

## Setup
First we need to import g2opy to be able to run.
We do this by adding the lib folder inside g2o's source to the python 
path and
afterwards import g2opy

In [ ]:
import os
import sys

import g2opy
import plotly.graph_objects as go

# Adjust path if needed
lib_directory = os.path.join(os.getcwd(), "../../build/lib")
print(f"lib_directory {lib_directory}")

if not lib_directory in sys.path:
    sys.path.append(lib_directory)

## Simulate 2D Data
We first create a simulator and specify its configuration.
Afterwards, we run a simulation.

In [ ]:
simulator = g2opy.Simulator2D()
simulator.config.has_odom = True
simulator.config.has_pose_sensor = True
simulator.config.world_size = 50.0
simulator.config.sim_steps = 500

simulator.setup()
simulator.simulate()

print("Simulation result")
print(f"Number of vertices: {len(simulator.graph().vertices())}")
print(f"Number of edges {len(simulator.graph().edges())}")

## Creating an optimizer and solver
Now we are ready to create an optimizer and a solver.
Furthermore, we add the simulated data into the optimizer.

In [ ]:
def create_optimizer():
    optimizer = g2opy.SparseOptimizer()
    solver = g2opy.BlockSolverX(g2opy.LinearSolverEigenX())
    solver = g2opy.OptimizationAlgorithmLevenberg(solver)
    optimizer.set_algorithm(solver)
    return optimizer


optimizer = create_optimizer()
optimizer.add_graph(simulator.graph())
print(f"Number of vertices: {len(optimizer.vertices())}")
print(f"Number of edges {len(optimizer.edges())}")

## Visualization of the initial state
First the generic code to visualize the graph by a plotly figure.

In [ ]:
def plot_slam2d(optimizer, title):
    def edges_coord(edge_ids, vertices_position, dim):
        for e in edge_ids:
            yield vertices_position[e[0]][dim]
            yield vertices_position[e[1]][dim]
            yield None

    fig = go.Figure()

    # position of the vertices
    vertices_position = {
        id: v.get_estimate_data()[0:2] for id, v in optimizer.vertices().items()
    }
    fig.add_trace(
        go.Scatter(
            x=[d[0] for d in vertices_position.values()],
            y=[d[1] for d in vertices_position.values()],
            mode="markers",
        )
    )

    # edge_ids
    edge_ids = [[v.id() for v in e.vertices()[0:2]] for e in optimizer.edges()]
    fig.add_trace(
        go.Scatter(
            x=list(edges_coord(edge_ids, vertices_position, 0)),
            y=list(edges_coord(edge_ids, vertices_position, 1)),
            mode="lines",
            line=dict(width=0.5),
        )
    )

    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )
    fig.update_layout(go.Layout({"title": title, "showlegend": False}))

    return fig

Below, we can visualize the data

In [ ]:
plot_slam2d(optimizer, "Initial state").show()

## Optimization and visualization
Run the optimization and visualize the result

In [ ]:
optimizer.vertices()[0].set_fixed(True)
optimizer.initialize_optimization()
optimizer.optimize(20)

plot_slam2d(optimizer, "Optimized state").show()